In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

import tensorflow as tf
import numpy as np
import time

from architecture import shufflenet

# add path to this:
# https://github.com/tensorflow/models
import sys
sys.path.append('../models/research/slim/')

from nets.mobilenet_v1 import mobilenet_v1
from nets.mobilenet.mobilenet_v2 import mobilenet

# Create a graph

In [ ]:
tf.reset_default_graph()

BATCH_SIZE = 8
random_batch = tf.constant(np.random.randn(BATCH_SIZE, 224, 224, 3), dtype=tf.float32)

# uncomment only one:
# logits, _ = mobilenet_v1(random_batch, is_training=False, depth_multiplier=1.0)
# logits, _ = mobilenet(random_batch, num_classes=1000, depth_multiplier=1.0)
logits = shufflenet(random_batch, is_training=False, num_classes=1000, depth_multiplier='2.0')

init = tf.global_variables_initializer()

# Measure inference times

In [ ]:
times = []

with tf.Session() as sess:
    sess.run(init)
    for _ in range(1010):
        start = time.perf_counter()
        sess.run(logits)
        times.append(time.perf_counter() - start)
        
times = np.array(times[10:])
print('mean:', times.mean(), 'std:', times.std())

# https://en.wikipedia.org/wiki/Taylor_expansions_for_the_moments_of_functions_of_random_variables
mean = times.mean()/BATCH_SIZE
std = times.std()/BATCH_SIZE
images_per_second = (1/mean) + (1/(mean**3))*(std**2)
images_per_second_std = std/(mean**2)

print('images per second:', images_per_second, images_per_second_std)